## Imports

In [1]:
import time

import numpy as np
import cv2 as cv
import os
import glob
import matplotlib.pyplot as plt
from cv2 import cvtColor
from numpy.random import uniform
import pdb
from tqdm import tqdm

import utils
from utils import show_image_matplot,show_image_cv

In [6]:
# utils.store_template_numbers1(10)
# utils.store_template_numbers2(8)

In [7]:
# utils.store_binary_templates(100,"./templates/sample1","./templates_binary/sample1")
# utils.store_binary_templates(100)

In [8]:
# new_templates = crop_and_store_templates()

In [9]:
#utils.rewrite_all()

In [3]:

# for i in range(4):
#     utils.move_train_data(i+1)


In [4]:
import mathable_solver as s

for i in range(0,4):
    solver = s.Solver(i+1,output_dest_dir="test")
    solver.solve()

100%|██████████| 50/50 [00:00<00:00, 179.69it/s]


In [3]:
os.makedirs("./343_Gabroveanu_Razvan", exist_ok=True)